# Function Generation for the Training of λ-Nets

## Package installation (uncommand first line to install packages at the beginning)

In [1]:
%%script false --no-raise-error
#if some errors occur during the installation consider using "sudo pip install"
!pip install numpy
!pip install pandas
!pip install truth-table-generator
!pip install more-itertools
!pip install tqdm
!pip install joblib
!pip install scipy
!pip install PrettyTable
!pip install colored
!pip install scikit-learn
!pip install keras
!pip install ipython
!pip install livelossplot
!pip install matplotlib
!pip install seaborn
!pip install tensorflow
!pip install tensorflow-gpu

## Specitication of Experiment Settings

In [2]:
import math
def nCr(n,r):
    f = math.factorial
    return f(n) // f(r) // f(n-r)

In [3]:
d = 3
n = 6
sparsity = nCr(n+d, d)


x_max = 1 
x_min = -1
x_step = 0.01
a_max = 10 
a_min = -10
a_step = 0.001

n_jobs = -3

lambda_dataset_size = 1000 #specify the number of data points to calculate the function values for (determines the lambda net training size)

interpretation_dataset_size = 100000 #specifies the number of functions generated (specifies the interpretation-net dataset size)

same_training_all_lambda_nets = False


In [4]:
##############DO NOT CHANGE###################
variables = 'abcdefghijklmnopqrstuvwxyz'[:n]

RANDOM_SEED = 42

if same_training_all_lambda_nets:
    training_string = '_same'
else:
    training_string = '_diverse'
    
print('Variables: ' + str(n) + ' (' + variables + ')')
print('Degree: ' + str(d))
print('Sparsity: ' + str(sparsity)) 
print('Lambda-Net Dataset Size: ' + str(lambda_dataset_size))
print('I-Net Dataset Size: ' + str(interpretation_dataset_size))
      
print('Coefficient Range: ' + '[' + str(a_min) + ', ' + str(a_max) + ']')
print('Variable Range: ' + '[' + str(x_min) + ', ' + str(x_max) + ']')

Variables: 6 (abcdef)
Degree: 3
Sparsity: 84
Lambda-Net Dataset Size: 1000
I-Net Dataset Size: 100000
Coefficient Range: [-10, 10]
Variable Range: [-1, 1]


## Imports

In [5]:
import ttg
from itertools import product       # forms cartesian products
from more_itertools import random_product 
from tqdm import tqdm_notebook as tqdm
import pickle

import numpy as np

import pandas as pd
from joblib import Parallel, delayed

import random 
from random import sample 
random.seed(42)

import os

directory_names = ['parameters', 'plotting', 'saved_polynomial_lists', 'results', 'saved_models', 'weights', 'weights_training']
if not os.path.exists('./data'):
    os.mkdir('./data')
for directory_name in directory_names:
    path = './data/' + directory_name
    if not os.path.exists(path):
        os.mkdir(path)
        
import seaborn as sns
sns.set_style("darkgrid")
        
import random 
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)




## Utility functions

In [6]:
#test for exact equality
def arreq_in_list(myarr, list_arrays):
    return next((True for elem in list_arrays if np.array_equal(elem, myarr)), False)

In [7]:
ALPHABET = \
  "0123456789abcdefghijklmnopqrstuvwxyz"

def encode (n):
    try:
        return ALPHABET [n]
    except IndexError:
        raise Exception ("cannot encode: %s" % n)

def dec_to_base (dec = 0, base = 16):
    if dec < base:
        return encode (dec)
    else:
        return dec_to_base (dec // base, base) + encode (dec % base)

# Function Generation

In [8]:
list_of_monomial_identifiers_extended = []
for i in tqdm(range((d+1)**n)):    
    monomial_identifier = dec_to_base(i, base = (d+1)).zfill(n) 
    list_of_monomial_identifiers_extended.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers_extended)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers_extended)

list_of_monomial_identifiers = []
for monomial_identifier in tqdm(list_of_monomial_identifiers_extended):
    monomial_identifier_values = list(map(int, list(monomial_identifier)))
    if sum(monomial_identifier_values) <= d:
        list_of_monomial_identifiers.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers)



List length: 4096
Number of monomials in a polynomial with 6 variables and degree 3: 84
Sparsity: 84
['000000', '000001', '000002', '000003', '000010', '000011', '000012', '000013', '000020', '000021', '000022', '000023', '000030', '000031', '000032', '000033', '000100', '000101', '000102', '000103', '000110', '000111', '000112', '000113', '000120', '000121', '000122', '000123', '000130', '000131', '000132', '000133', '000200', '000201', '000202', '000203', '000210', '000211', '000212', '000213', '000220', '000221', '000222', '000223', '000230', '000231', '000232', '000233', '000300', '000301', '000302', '000303', '000310', '000311', '000312', '000313', '000320', '000321', '000322', '000323', '000330', '000331', '000332', '000333', '001000', '001001', '001002', '001003', '001010', '001011', '001012', '001013', '001020', '001021', '001022', '001023', '001030', '001031', '001032', '001033', '001100', '001101', '001102', '001103', '001110', '001111', '001112', '001113', '001120', '001121


List length: 84
Number of monomials in a polynomial with 6 variables and degree 3: 84
Sparsity: 84
['000000', '000001', '000002', '000003', '000010', '000011', '000012', '000020', '000021', '000030', '000100', '000101', '000102', '000110', '000111', '000120', '000200', '000201', '000210', '000300', '001000', '001001', '001002', '001010', '001011', '001020', '001100', '001101', '001110', '001200', '002000', '002001', '002010', '002100', '003000', '010000', '010001', '010002', '010010', '010011', '010020', '010100', '010101', '010110', '010200', '011000', '011001', '011010', '011100', '012000', '020000', '020001', '020010', '020100', '021000', '030000', '100000', '100001', '100002', '100010', '100011', '100020', '100100', '100101', '100110', '100200', '101000', '101001', '101010', '101100', '102000', '110000', '110001', '110010', '110100', '111000', '120000', '200000', '200001', '200010', '200100', '201000', '210000', '300000']


In [9]:
list_of_polynomials = []

if interpretation_dataset_size/((a_max-a_min)*10**int(-np.log10(a_step)))**(nCr(n+d, d)) <= 10e-4:
    while len(list_of_polynomials) < interpretation_dataset_size:
        random_polynomial = list(random_product([i*a_step for i in range(int(a_min*10**int(-np.log10(a_step))), int(a_max*10**int(-np.log10(a_step))))], repeat=nCr(n+d, d)))
        if random_polynomial not in list_of_polynomials:
            list_of_polynomials.append(random_polynomial)
else:
    all_polynomials_list = list(product([i*a_step for i in range(int(a_min*10**int(-np.log10(a_step))), int(a_max*10**int(-np.log10(a_step))))], repeat=nCr(n+d, d)))
    list_of_polynomials = [all_polynomials_list[i] for i in np.random.choice(len(all_polynomials_list), interpretation_dataset_size, replace=False)]
    del all_polynomials_list

for polynomial in tqdm(list_of_polynomials):
    sparsity_indices = np.random.choice(nCr(n+d, d), nCr(n+d, d)-sparsity, replace=False)
    for sparsity_index in sparsity_indices:                            
        polynomial[sparsity_index] = 0
                                
polynomials_list_df = pd.DataFrame(data=list_of_polynomials, columns=list_of_monomial_identifiers)

    
print(len(list_of_monomial_identifiers))
print(polynomials_list_df.shape)


84
(100000, 84)


In [10]:
polynomials_list_df.head()

,000000,000001,000002,000003,000010,000011,000012,000020,000021,000030,...,110100,111000,120000,200000,200001,200010,200100,201000,210000,300000
0,-6.352,-9.181,-0.988,-1.976,-2.686,-5.428,-6.642,7.870,-7.152,9.349,...,-1.979,-4.646,5.147,2.433,-1.155,8.250,-2.804,0.626,-8.168,-2.495
1,-8.949,0.336,3.145,-1.227,-7.832,-3.087,8.585,0.311,-3.033,6.358,...,1.894,0.076,-2.154,-8.102,-2.108,8.591,-7.420,-7.194,5.924,-7.733
2,7.455,-5.880,-5.793,5.574,8.015,-4.590,-1.315,7.290,9.876,3.865,...,5.159,-1.815,-7.530,4.520,8.033,-6.792,-8.343,7.713,-9.517,-6.944
3,-2.255,-4.551,3.317,5.913,5.773,-2.996,3.141,-8.079,-4.606,2.418,...,-1.310,-5.660,1.436,-7.746,-1.996,2.108,-0.662,-4.831,4.358,7.800
4,-0.088,7.332,-9.744,8.173,-0.190,-6.606,-5.600,-1.334,-6.218,-6.493,...,5.423,-2.712,-3.461,5.083,1.457,0.000,-2.543,-2.696,-9.225,-3.672


In [11]:
def generate_random_x_values(size, x_max, x_min, x_step, numnber_of_variables):
    x_values_list = []
    
    for j in range(size):
        values = np.round(np.array(random_product(np.arange(x_min, x_max, x_step), repeat=numnber_of_variables)), int(-np.log10(x_step)))
        while arreq_in_list(values, x_values_list):
                values = np.round(np.array(random_product(np.arange(x_min, x_max, x_step), repeat=numnber_of_variables)), int(-np.log10(x_step)))         
        x_values_list.append(values)
    
    return x_values_list


In [12]:
if same_training_all_lambda_nets:
    x_values = generate_random_x_values(lambda_dataset_size, x_max, x_min, x_step, n)
    x_values_list = [x_values for _ in range(interpretation_dataset_size)]
else:
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    x_values_list = parallel(delayed(generate_random_x_values)(lambda_dataset_size, x_max, x_min, x_step, n) for _ in range(interpretation_dataset_size))
    del parallel


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-3)]: Done 1108 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-3)]: Done 1524 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-3)]: Done 2004 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-3)]: Done 2548 tasks      | elapsed: 15.3min
[Parallel(n_jobs=-3)]: Done 3156 tasks      | elapsed: 18.9min
[Parallel(n_jobs=-3)]: Done 3828 tasks      | elapsed: 22.9min
[Parallel(n_jobs=-3)]: Done 4564 tasks      | elapsed: 27.3min
[Parallel(n_jobs=-3)]: Done 5364 tasks      | elapsed: 32.1min
[Parallel(n_jobs=-3)]: Done 6228 tasks      | elapsed: 37.2min
[Parallel(n_jobs=-3)]: Done 7156 tasks      | elapsed: 42.7min
[Parallel(n_jobs=-3)]: Done 8148 tasks      |

In [13]:
from functools import reduce

def calcualate_function_with_data(coefficient_list, variable_values):
    
    result = 0    
    for coefficient_value, coefficient_multipliers in zip(coefficient_list, list_of_monomial_identifiers):
        partial_results = [variable_value**int(coefficient_multiplier) for coefficient_multiplier, variable_value in zip(coefficient_multipliers, variable_values)]
        
        result += coefficient_value * reduce(lambda x, y: x*y, partial_results)

    return result, variable_values
 
def calculate_function_values_from_polynomial(true_value_test, evaluation_dataset):
    if isinstance(true_value_test, pd.DataFrame):
        true_value_test = true_value_test.values
        
    true_value_fv = []
    true_value_coeff = []
    
    for evaluation in evaluation_dataset:
        true_function_value, true_coeff = calcualate_function_with_data(true_value_test, evaluation)
       
        true_value_fv.append(true_function_value) 
        true_value_coeff.append(true_coeff)
        
    return [true_value_test, pd.DataFrame(np.array(true_value_coeff))], [true_value_test, pd.DataFrame(np.array(true_value_fv))]


In [14]:
result_list = []

polynomials_X_data_list = []
polynomials_y_data_list = []
    
chunks = max(interpretation_dataset_size//1000, 1)

for polynomials_list_df_chunk in tqdm(np.array_split(polynomials_list_df, chunks), total=chunks):
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_sublist = parallel(delayed(calculate_function_values_from_polynomial)(polynomial, x_values) for (_, polynomial), x_values in zip(polynomials_list_df_chunk.iterrows(), x_values_list))  
    result_list.extend(result_sublist)
    del parallel

polynomials_X_data_list = [result[0] for result in result_list]
polynomials_y_data_list = [result[1] for result in result_list]


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   45.1s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.5min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   40.2s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.4min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.6s
[Pa

[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   59.0s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   43.5s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.4min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   36.7s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   58.5s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.4min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent w

[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   59.3s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.5min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   37.1s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   59.0s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   37.1s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   59.2s
[Parallel(n_jobs=-

[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   37.4s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   59.7s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   37.1s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   59.3s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.4min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   36.3s
[Parallel(n_jobs=-

[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   37.1s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   58.8s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   36.3s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   58.7s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed: 

[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished


In [15]:
polynomials_X_data_list[0][0].head()

000000   -6.352
000001   -9.181
000002   -0.988
000003   -1.976
000010   -2.686
Name: 0, dtype: float64

In [16]:
polynomials_X_data_list[0][1].head()

,0,1,2,3,4,5
0,-0.48,-0.32,-0.01,0.42,-0.96,0.74
1,0.30,0.38,0.83,-0.47,-0.98,-0.36
2,0.28,0.74,0.77,-0.13,0.05,-0.78
3,0.60,-0.48,-0.10,0.43,0.39,0.61
4,-0.01,0.13,-0.47,-0.06,0.38,-0.18


In [17]:
polynomials_y_data_list[0][0].values

array([-6.352, -9.181, -0.988, -1.976, -2.686, -5.428, -6.642,  7.87 ,
       -7.152,  9.349,  3.825, -8.959, -9.024, -6.93 , -2.836, -2.377,
        6.559,  9.726, -9.131,  8.39 , -3.485,  7.856,  3.746, -2.777,
        4.719,  9.309, -0.885, -9.788, -4.769,  3.848,  1.149, -0.895,
       -4.906, -2.945,  1.029, -6.651, -6.961,  2.449, -6.831,  1.763,
        1.27 ,  9.782, -1.333, -8.577,  5.054,  7.571, -5.91 ,  2.403,
       -7.418,  8.089, -0.394,  1.85 ,  8.918, -3.7  , -7.721, -8.499,
       -2.533, -0.518, -7.386, -2.372, -6.691,  2.455, -0.892,  4.857,
        1.954, -4.671,  2.13 ,  1.641, -3.135, -1.252, -7.661,  9.96 ,
       -4.393,  7.502, -1.979, -4.646,  5.147,  2.433, -1.155,  8.25 ,
       -2.804,  0.626, -8.168, -2.495])

In [18]:
polynomials_y_data_list[0][0].head()

000000   -6.352
000001   -9.181
000002   -0.988
000003   -1.976
000010   -2.686
Name: 0, dtype: float64

In [19]:
polynomials_y_data_list[0][1].head()

,0
0,-10.633011
1,8.344392
2,-1.581632
3,-15.106706
4,-3.047740


In [20]:
polynomials_X_data_list[0]

[000000   -6.352
 000001   -9.181
 000002   -0.988
 000003   -1.976
 000010   -2.686
 000011   -5.428
 000012   -6.642
 000020    7.870
 000021   -7.152
 000030    9.349
 000100    3.825
 000101   -8.959
 000102   -9.024
 000110   -6.930
 000111   -2.836
 000120   -2.377
 000200    6.559
 000201    9.726
 000210   -9.131
 000300    8.390
 001000   -3.485
 001001    7.856
 001002    3.746
 001010   -2.777
 001011    4.719
 001020    9.309
 001100   -0.885
 001101   -9.788
 001110   -4.769
 001200    3.848
           ...  
 021000   -7.721
 030000   -8.499
 100000   -2.533
 100001   -0.518
 100002   -7.386
 100010   -2.372
 100011   -6.691
 100020    2.455
 100100   -0.892
 100101    4.857
 100110    1.954
 100200   -4.671
 101000    2.130
 101001    1.641
 101010   -3.135
 101100   -1.252
 102000   -7.661
 110000    9.960
 110001   -4.393
 110010    7.502
 110100   -1.979
 111000   -4.646
 120000    5.147
 200000    2.433
 200001   -1.155
 200010    8.250
 200100   -2.804
 201000    0.6

In [21]:
polynomials_y_data_list[0]

[000000   -6.352
 000001   -9.181
 000002   -0.988
 000003   -1.976
 000010   -2.686
 000011   -5.428
 000012   -6.642
 000020    7.870
 000021   -7.152
 000030    9.349
 000100    3.825
 000101   -8.959
 000102   -9.024
 000110   -6.930
 000111   -2.836
 000120   -2.377
 000200    6.559
 000201    9.726
 000210   -9.131
 000300    8.390
 001000   -3.485
 001001    7.856
 001002    3.746
 001010   -2.777
 001011    4.719
 001020    9.309
 001100   -0.885
 001101   -9.788
 001110   -4.769
 001200    3.848
           ...  
 021000   -7.721
 030000   -8.499
 100000   -2.533
 100001   -0.518
 100002   -7.386
 100010   -2.372
 100011   -6.691
 100020    2.455
 100100   -0.892
 100101    4.857
 100110    1.954
 100200   -4.671
 101000    2.130
 101001    1.641
 101010   -3.135
 101100   -1.252
 102000   -7.661
 110000    9.960
 110001   -4.393
 110010    7.502
 110100   -1.979
 111000   -4.646
 120000    5.147
 200000    2.433
 200001   -1.155
 200010    8.250
 200100   -2.804
 201000    0.6

In [22]:
path_polynomials = './data/saved_polynomial_lists/polynomials_sample' + str(interpretation_dataset_size) + '_variables_' + str(n) +  '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '.csv'
polynomials_list_df.to_csv(path_polynomials, index=False)

path_X_data = './data/saved_polynomial_lists/X_sample' + str(interpretation_dataset_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step) + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '.pkl'
with open(path_X_data, 'wb') as f:
    pickle.dump(polynomials_X_data_list, f)#, protocol=2)
    
path_y_data = './data/saved_polynomial_lists/y_sample' + str(interpretation_dataset_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step) + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '.pkl'
with open(path_y_data, 'wb') as f:
    pickle.dump(polynomials_y_data_list, f)#, protocol=2)
